### Imports

In [104]:
#Time
import time
from datetime import datetime


#Random
import random

#CSV
import csv  

#OS
import os

# Selenium
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.firefox.options import Options

#Scalability
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import Process
from concurrent.futures import ThreadPoolExecutor

#Seyed
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import trafilatura
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import reuters
import numpy as np
import pandas as pd
from collections import Counter


### CSV

In [93]:
def saveCsv(
    type,
    source,
    url,
    id,
    topic,
    author,
    timeRelease,
    vote,
    commentsCount,
    language,
    token,
    predicte,
    rawText,
):
    columnTopics = [
        "Type",
        "Source",
        "URL",
        "ID",
        "Topic",
        "Author",
        "TimeRelease",
        "Vote",
        "CommentsCount",
        "Language",
        "Token",
        "Predicte",
        "RawText",
    ]
    row = [
        type,
        source,
        url,
        id,
        topic,
        author,
        timeRelease,
        vote,
        commentsCount,
        language,
        token,
        predicte,
        rawText
    ]
    
    fileName = "RecordsNewsReddit.csv"
    file_exists = os.path.exists(fileName)
    
    with open(fileName, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        
        if not file_exists:
            writer.writerow(columnTopics)
        
        writer.writerow(row)

In [94]:
""" #Example Save Data
saveCsv(
    type="Reddit",
    source="r/news",
    url="https://reddit.com/r/news/xyz",
    id="abc123",
    topic="Breaking News",
    author="user42",
    timeRelease="2025-09-08 12:00",
    vote=420,
    commentsCount=69,
    language="en",
    token="breaking,news",
    predicte="Politics"
) """

' #Example Save Data\nsaveCsv(\n    type="Reddit",\n    source="r/news",\n    url="https://reddit.com/r/news/xyz",\n    id="abc123",\n    topic="Breaking News",\n    author="user42",\n    timeRelease="2025-09-08 12:00",\n    vote=420,\n    commentsCount=69,\n    language="en",\n    token="breaking,news",\n    predicte="Politics"\n) '

### Reddit Scrapper

init -> (userAgent)(no need)
→ (login)(Optional)
→ loop { 
     rateLimiter
     randomDelay
     fetch/searchSubReddit
     cleanData(deduplication + canonicalization)
     saveData(incremental)
  }

In [95]:
#init
def init():
    try:
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:142.0) Gecko/20100101 Firefox/142.0",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 15.6; rv:142.0) Gecko/20100101 Firefox/142.0",
            "Mozilla/5.0 (X11; Linux i686; rv:142.0) Gecko/20100101 Firefox/142.0",
            "Mozilla/5.0 (X11; Linux x86_64; rv:142.0) Gecko/20100101 Firefox/142.0",
            "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:142.0) Gecko/20100101 Firefox/142.0"
        ]
        random_agent = random.choice(user_agents)

        options = Options()
        options.set_preference("general.useragent.override", random_agent)

        service = Service("/usr/bin/geckodriver")
        driver = webdriver.Firefox(service=service, options=options)
        
        print(f"[INFO] Using User-Agent: {random_agent}")
        return driver

    except WebDriverException as e:
        print("WebDriver error happened:", e)
        return None

In [73]:
#randomDelay
def randomDelay():
    time.sleep(random.uniform(2,5))

In [75]:
def cleanData(type, source, post_id, title, author, votes, comments, time_release, url):
    """Clean, deduplicate, canonicalize, and save post."""
    fileName = "RecordsNewsReddit.csv"
    
    # Canonicalize: strip strings
    post_id = post_id.strip() if post_id else "N/A"
    title = title.strip() if title else "N/A"
    author = author.strip() if author else "N/A"
    url = url.strip() if url else "N/A"
    
    # Convert votes/comments to int if possible
    try:
        votes = int(votes)
    except:
        votes = 0
    try:
        comments = int(comments)
    except:
        comments = 0
    
    # Convert time_release to standard format
    try:
        if time_release != "N/A":
            dt = datetime.fromisoformat(time_release.replace("Z", "+00:00"))
            time_release = dt.strftime("%Y-%m-%d %H:%M:%S")
        else:
            time_release = "N/A"
    except:
        time_release = "N/A"
    
    # Deduplication: check existing CSV
    existing_ids = set()
    if os.path.isfile(fileName):
        with open(fileName, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_ids.add(row["ID"])
    
    if post_id in existing_ids:
        print(f"Duplicate found: {post_id}, skipping...")
        return
    
    # Save to CSV
    saveCsv(
        type=type,
        source=source,
        url=url,
        id=post_id,
        topic=title,
        author=author,
        timeRelease=time_release,
        vote=votes,
        commentsCount=comments,
        language="N/A",
        token="N/A",
        predicte="N/A",
        rawText="N/A"
    )

Xpath

/html/body/shreddit-app/div[3]/div/div[2]/main/div[2]/shreddit-feed/article[1]/shreddit-post/a

/html/body/shreddit-app/div[3]/div/div[2]/main/div[2]/shreddit-feed/article[2]/shreddit-post/a

=====================

CSS Selector:
#t3_1nc1j6c > a:nth-child(1)
#t3_1n43yfc > a:nth-child(1)

=====================

[json for reddit](https://www.reddit.com/r/news/.json)

-----------------------
(https://www.reddit.com/r/news/comments/1mvlhxn/texas_cant_require_the_ten_commandments_in_every/)

In [76]:
def extractDataSubReddit(driver, type, source, total_posts=50, posts_before_cooldown=30, cooldown_time=60):
    scraped = 0
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Track last time we saved a post
    last_scrape_time = time.time()

    while scraped < total_posts:
        # Check if 3 minutes passed without scraping new posts
        if time.time() - last_scrape_time > 180:
            print("No new posts saved for 3 minutes. Stopping...")
            break

        try:
            # Wait until at least one post is loaded
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "shreddit-post"))
            )
            posts = driver.find_elements(By.CSS_SELECTOR, "shreddit-post")

            # If next post is not loaded, scroll a bit and wait
            if scraped >= len(posts):
                driver.execute_script("window.scrollBy(0, 800);")  # small scroll
                time.sleep(random.uniform(1, 2))
                continue

            post = posts[scraped]

        except Exception as e:
            print(f"No posts loaded yet or error: {e}")
            time.sleep(2)
            continue

        # Extract attributes safely
        post_id = post.get_attribute("id") or "N/A"
        permalink = post.get_attribute("permalink") or "N/A"
        title = post.get_attribute("post-title") or "N/A"
        author = post.get_attribute("author") or "N/A"
        votes = post.get_attribute("score") or "0"
        comments = post.get_attribute("comment-count") or "0"
        time_release = post.get_attribute("created-timestamp") or "N/A"
        url = post.get_attribute("content-href") or "N/A"

        # Clean, deduplicate, and save
        cleanData(type, source, post_id, title, author, votes, comments, time_release, url)

        scraped += 1
        last_scrape_time = time.time()  # reset timer since we got a new post
        time.sleep(random.uniform(1, 2))

        # Cooldown after N posts
        if scraped % posts_before_cooldown == 0:
            print(f"Cooldown: {scraped} posts scraped. Waiting {cooldown_time} sec...")
            time.sleep(cooldown_time)

        # Scroll if at bottom of loaded content
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(random.uniform(1, 2))
            new_height = driver.execute_script("return document.body.scrollHeight")
        last_height = new_height


In [77]:
#subReddits List
subRedditsNews = [
    "r/news/",
    "r/worldnews/",
    "r/realbbcnews/",
    "r/world24x7hr/"
] 

In [78]:
#fetch/searchSubReddit
def searchRedditNews(subRedditsNews, total_posts_per_subreddit=1):
    baseURLReddit = "https://www.reddit.com/"
    driver = init()
    for subreddit in subRedditsNews:   
        full_url = baseURLReddit + subreddit     
        driver.get(full_url) 
        clean_url = driver.execute_script("return window.location.origin + window.location.pathname")
        driver.get(clean_url)        
        type = "reddit"
        source = subreddit
        randomDelay()       
        extractDataSubReddit(driver, type, source, total_posts=total_posts_per_subreddit)
        time.sleep(10)
    driver.quit()
    

In [96]:
searchRedditNews(subRedditsNews)

[INFO] Using User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:142.0) Gecko/20100101 Firefox/142.0
Duplicate found: t3_1ncuh2y, skipping...
Duplicate found: t3_1n7miac, skipping...
Duplicate found: t3_1n9epz3, skipping...


### Extract text from each news

In [97]:
def get_raw_text(url):
    """Fetch raw text from a URL using requests + BeautifulSoup."""
    # Skip image URLs
    if any(x in url for x in ["i.redd.it", "redd.it", "reddit"]):
        return "N/A"
    
    try:
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/116.0.0.0 Safari/537.36"
            )
        }
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch {url}, status code: {response.status_code}")
            return "N/A"

        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p")
        raw_text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
        if not raw_text:
            return "N/A"
        return raw_text

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return "N/A"



In [98]:
def scrapeRawTextFromCSV(csv_file="RecordsNewsReddit.csv"):
    """Scrape RawText from URLs in a CSV using requests + BeautifulSoup."""
    if not os.path.isfile(csv_file):
        print(f"CSV file {csv_file} not found.")
        return

    df = pd.read_csv(csv_file, keep_default_na=False)

    # Create RawText column if it doesn't exist
    if "RawText" not in df.columns:
        df["RawText"] = ""

    for idx, row in df.iterrows():
        if str(row.get("Type", "")).lower() != "reddit":
            continue

        raw_text_val = str(row.get("RawText", ""))
        if raw_text_val.strip() not in ["", "N/A"]:
            # Already scraped
            continue

        url = row.get("URL", "")
        if not isinstance(url, str) or not url.startswith("http"):
            print(f"Skipping invalid URL: {url}")
            df.at[idx, "RawText"] = "N/A"
            continue

        print(f"Scraping raw text from: {url}")
        raw_text = get_raw_text(url)
        df.at[idx, "RawText"] = raw_text

        # Small delay to avoid overloading servers    
        # Save updated CSV
        df.to_csv(csv_file, index=False, encoding="utf-8") 
        print(f"we are saving {row.get('ID','')}.")   
        time.sleep(1)
        
    print(f"CSV updated in-place: {csv_file}")




In [99]:
scrapeRawTextFromCSV()

Scraping raw text from: https://www.kiro7.com/news/local/wa-congressional-candidates-husband-military-veteran-taken-into-ice-custody/AGN6756IA5F7ZHHEQYRSIXWAOM/?outputType=amp
we are saving t3_1mz25ee.
Scraping raw text from: https://newhampshirebulletin.com/2025/08/14/number-of-canadian-tourists-visiting-new-hampshire-down-30-this-year-officials-say/
Failed to fetch https://newhampshirebulletin.com/2025/08/14/number-of-canadian-tourists-visiting-new-hampshire-down-30-this-year-officials-say/, status code: 403
we are saving t3_1mrxs29.
Scraping raw text from: https://www.france24.com/en/france/20250826-france-to-sue-australian-platform-kick-for-negligence-after-livestream-death
Failed to fetch https://www.france24.com/en/france/20250826-france-to-sue-australian-platform-kick-for-negligence-after-livestream-death, status code: 403
we are saving t3_1n0mds2.
Scraping raw text from: https://www.nytimes.com/2025/08/26/us/politics/doge-social-security-data.html?unlocked_article_code=1.hE8.Zz

### Scalability

Multi Process For RedditScrap
<br/>
Mult Thread For News Crawl Text 

In [102]:
def worker(sublist, total_posts_per_subreddit=300):
        searchRedditNews(sublist, total_posts_per_subreddit=total_posts_per_subreddit)

def multiProcessReddit(subRedditsNews):
    mid = len(subRedditsNews) // 2
    half1 = subRedditsNews[:mid]
    half2 = subRedditsNews[mid:]

    p1 = Process(target=worker, args=(half1,))
    p2 = Process(target=worker, args=(half2,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()
    print("All subreddits scraped!")


In [103]:
multiProcessReddit(subRedditsNews)

[INFO] Using User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:142.0) Gecko/20100101 Firefox/142.0
[INFO] Using User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:142.0) Gecko/20100101 Firefox/142.0
Duplicate found: t3_1ncl6mv, skipping...
Duplicate found: t3_1nd3ffp, skipping...
Duplicate found: t3_1nd7bca, skipping...
Duplicate found: t3_1n61jbg, skipping...
Duplicate found: t3_1mwho1l, skipping...
Duplicate found: t3_1n43yfc, skipping...
Duplicate found: t3_1nchhym, skipping...
Duplicate found: t3_1nd0ta7, skipping...
Duplicate found: t3_1n8a591, skipping...
Duplicate found: t3_1mu1urk, skipping...
Duplicate found: t3_1nc2stv, skipping...
Duplicate found: t3_1nbu4fg, skipping...
Duplicate found: t3_1mzliak, skipping...
Duplicate found: t3_1nd701g, skipping...
Duplicate found: t3_1n1yaro, skipping...
Duplicate found: t3_1nbum6b, skipping...
Duplicate found: t3_1nc9te8, skipping...
Duplicate found: t3_1n7tbtq, skipping...
Duplicate found: t3_1nc5i0c, skipping...
Duplicate found: t3_1nb

In [105]:
def scrape_row(idx, row, csv_file):
    if str(row.get("Type", "")).lower() != "reddit":
        return None, None

    raw_text_val = str(row.get("RawText", ""))
    if raw_text_val.strip() not in ["", "N/A"]:
        return idx, raw_text_val  # already scraped

    url = row.get("URL", "")
    if not isinstance(url, str) or not url.startswith("http"):
        print(f"Skipping invalid URL: {url}")
        return idx, "N/A"

    print(f"Scraping raw text from: {url}")
    raw_text = get_raw_text(url)
    time.sleep(1)  # avoid hitting server too fast
    return idx, raw_text


In [106]:
def scrapeRawTextFromCSV_multithreaded(csv_file="RecordsNewsReddit.csv", max_workers=5):
    if not os.path.isfile(csv_file):
        print(f"CSV file {csv_file} not found.")
        return

    df = pd.read_csv(csv_file, keep_default_na=False)
    if "RawText" not in df.columns:
        df["RawText"] = ""

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(scrape_row, idx, row, csv_file) for idx, row in df.iterrows()]

        for future in futures:
            idx, raw_text = future.result()
            if idx is not None:
                df.at[idx, "RawText"] = raw_text
                # save incrementally
                df.to_csv(csv_file, index=False, encoding="utf-8")
                print(f"Saved {df.at[idx, 'ID']}")

    print(f"CSV updated in-place: {csv_file}")

In [107]:
scrapeRawTextFromCSV_multithreaded(csv_file="RecordsNewsReddit.csv", max_workers=5)

Scraping raw text from: https://newhampshirebulletin.com/2025/08/14/number-of-canadian-tourists-visiting-new-hampshire-down-30-this-year-officials-say/
Scraping raw text from: https://www.france24.com/en/france/20250826-france-to-sue-australian-platform-kick-for-negligence-after-livestream-death
Scraping raw text from: https://www.nytimes.com/2025/08/26/us/politics/doge-social-security-data.html?unlocked_article_code=1.hE8.Zz4X.KRumBN4DjNR1&smid=nytcore-ios-share&referringSource=articleShare
Scraping raw text from: https://www.reuters.com/world/americas/canada-remove-many-retaliatory-tariffs-us-goods-says-carney-2025-08-22/
Scraping raw text from: https://www.reuters.com/business/media-telecom/nineteen-killed-nepal-gen-z-protest-over-social-media-ban-corruption-2025-09-08/
Saved t3_1ncel4h
Saved t3_1n43yfc
Saved t3_1nc1j6c
Saved t3_1n9t9jw
Saved t3_1n7ps3n
Saved t3_1mzozwd
Saved t3_1n4x2fo
Saved t3_1n3h1yn
Saved t3_1mr8qcq
Saved t3_1n1p32b
Saved t3_1n1qwje
Saved t3_1mz25ee
Saved t3_1n6

### Web Crawler

In [18]:
news_seeds = [
    "https://www.theguardian.com/world", # 54
    "https://www.aljazeera.com/news/", # 19
    "https://www.nytimes.com/section/world",# forbidden url
    "https://www.nbcnews.com/world",# keyword: world , 22
]

visited = set()
articles = []

In [19]:
def extract_text(html, url):
    """Try to extract main article text using trafilatura, fallback to <p> tags."""
    text = trafilatura.extract(html, url=url)
    if text:
        return text
    else:
        soup = BeautifulSoup(html, "html.parser")
        return " ".join([p.get_text() for p in soup.find_all("p")])

def crawl(url, index, depth=1):
    if depth == 0 or url in visited:
        return
    
    if True:
        print(f"Crawling: {url}")
        visited.add(url)
        
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            return
        
        html = response.text
        
        text = extract_text(html, url)
        
        if text:
            if "2025" in url:
                articles.append({"url": url, "id": 0, "title": 0, "writer": 0,  "raw_text": text})
                print(f"Saved article ({len(text)} chars)")
        else:
            print(f"Not an article, but checking links...")
        
        # Parse links
        soup = BeautifulSoup(html, "html.parser")
        for link in soup.find_all("a", href=True):
            new_url = urljoin(url, link["href"])
            
            # Filter: must be within same domain as seed
            if new_url not in visited and url.split("/")[2] in new_url:
                crawl(new_url, index, depth - 1)


crawl(news_seeds[0], 0, depth=2)

print(f"Collected {len(articles)} articles")

Crawling: https://www.theguardian.com/world
Crawling: https://www.theguardian.com/world#maincontent
Crawling: https://www.theguardian.com/world#navigation
Crawling: https://www.theguardian.com/email-newsletters
Crawling: https://www.theguardian.com/preference/edition/us
Crawling: https://www.theguardian.com/preference/edition/uk
Crawling: https://www.theguardian.com/preference/edition/au
Crawling: https://www.theguardian.com/preference/edition/eur
Crawling: https://www.theguardian.com/preference/edition/int
Crawling: https://www.theguardian.com/
Crawling: https://www.theguardian.com/commentisfree
Crawling: https://www.theguardian.com/sport
Crawling: https://www.theguardian.com/culture
Crawling: https://www.theguardian.com/lifeandstyle
Crawling: https://www.theguardian.com/us-news
Crawling: https://www.theguardian.com/us-news/us-politics
Crawling: https://www.theguardian.com/environment/climate-crisis
Crawling: https://www.theguardian.com/world/middleeast
Crawling: https://www.theguardi

In [20]:
print(articles[10]['raw_text'])

The federal government is racing to save a major new agreement with Vanuatu, after Anthony Albanese’s plans to sign the deal were rebuffed over concerns about infrastructure funding from China.
Speaking alongside Vanuatu’s prime minister, Jotham Napat, on Tuesday, Albanese said he was confident the Nakamal agreement will be “able to be signed soon”, talking up cooperation and proper process with Vanuatu’s governing coalition.
The Australian prime minister travelled to Port Vila before this week’s Pacific Islands Forum, hoping to sign the agreement, which would see Australia spend up to $500m over a decade on climate change resilience, security services and the economy.
Sign up: AU Breaking News email
It is designed to lock in Australia as Vanuatu’s primary security partner and to push back against China’s efforts to build ties across the Pacific, including through funding for major infrastructure projects.
“It’s important that processes be respected and I respect the prime minister’s p

### Preprocessing

In [21]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("reuters")

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Downloading package punkt to
[nltk_data]     /home/AmirhosseinRaeghi-400522373/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/AmirhosseinRaeghi-400522373/nltk_data...
[nltk_data] Downloading package reuters to
[nltk_data]     /home/AmirhosseinRaeghi-400522373/nltk_data...


True

In [22]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/AmirhosseinRaeghi-400522373/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/share/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [23]:
def normalize(articles):
    for article in articles:
        text = article['raw_text']
        text = text.lower()
        text = text.replace("\n", " ")

        # Remove URLs
        text = re.sub(r'http\S+|www\.\S+', '', text)
        # Remove special characters (keep only words and numbers)
        text = re.sub(r'[^a-z0-9\s]', ' ', text)

        article['normalize_text'] = text

# normalize(articles)
# print(articles[10]['normalize_text'])

In [24]:
def tokenize(articles):
    for article in articles:
        tokens = nltk.word_tokenize(article['normalize_text'])
    
        # Remove stopwords
        article["tokens"] = [t for t in tokens if t not in stop_words] 

# tokenize(articles)
# print(articles[10]['tokens'])

In [25]:
def stem(articles):
    for article in articles:
        article["tokens"] = [stemmer.stem(t) for t in article["tokens"]]

# stem(articles)
# print(articles[10]['tokens'])

In [26]:
def lemmatize(articles):
    for article in articles:
        article["tokens"] = [lemmatizer.lemmatize(t) for t in article["tokens"]]

# lemmatize(articles)
# print(articles[10]['tokens'])

In [27]:
def preprocess(articles):
    normalize(articles)
    tokenize(articles)
    stem(articles)
    # lemmatize(articles)
    
preprocess(articles)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/AmirhosseinRaeghi-400522373/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/share/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
***************************************************************

### Classification

In [28]:
dataset_size = 200000

dataset_articles = []
for fileid in reuters.fileids()[:dataset_size]:
    text = reuters.raw(fileid)
    cats = reuters.categories(fileid)  # some docs have multiple categories
    if cats:
        dataset_articles.append({"raw_text": text, "label": cats[0]})

dataset_articles[10]["label"]

'tin'

In [29]:
preprocess(dataset_articles)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/home/AmirhosseinRaeghi-400522373/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/share/nltk_data'
    - '/home/AmirhosseinRaeghi-400522373/Amirhossein life/Uni/term8/webSearch/Project/Web-News-Classifier/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
***************************************************************

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_reuters = vectorizer.fit_transform([" ".join(dataset_article["tokens"]) for dataset_article in dataset_articles])
X_crawled = vectorizer.transform([" ".join(article["tokens"]) for article in articles])

In [ ]:
category_vocab = {}
labels = [dataset_article["label"] for dataset_article in dataset_articles]
labels = list(set(labels))  # unique labels

for label in labels:
    # find indices of docs belonging to this label
    idx = [i for i, art in enumerate(dataset_articles) if art["label"] == label]
    
    if not idx:  # skip empty categories
        continue
    
    # average TF-IDF weights for this category
    avg_weights = np.asarray(X_reuters[idx].mean(axis=0)).ravel()
    
    # get indices of top 50 highest-weighted terms
    top_idx = avg_weights.argsort()[::-1][:50]
    
    # get actual term strings
    terms = vectorizer.get_feature_names_out()[top_idx]
    
    # get their corresponding weights
    weights = avg_weights[top_idx]
    
    # save into dictionary
    category_vocab[label] = dict(zip(terms, weights))

In [ ]:
def score_article(article_vector):
    scores = {}
    feature_names = vectorizer.get_feature_names_out()
    vec_array = article_vector.toarray().ravel()
    for label, vocab in category_vocab.items():
        score = sum(vec_array[feature_names.tolist().index(term)] * w
                    for term, w in vocab.items() if term in feature_names)
        scores[label] = score
    return scores

for article, x in zip(articles, X_crawled):
    scores = score_article(x)  # returns a dict of {label: score}
    article["scores"] = scores
    article["predicted"] = max(scores, key=scores.get)


In [ ]:
articles[10]["predicted"]
print(Counter(a["predicted"] for a in articles))



In [ ]:
for article in articles[:5]:
    print(article["raw_text"])
    print(f"### predicted class: {article['predicted']}")
    print("*"*20)